This notebook is used to evaluate ragas metric for combine approach (RAG + Finetuning)

In [1]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

# Load the model and processor
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa", # Use PyTorch's native optimized attention
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_id)

/workspace/eval_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 729/729 [00:11<00:00, 60.79it/s, Materializing param=model.visual.patch_embed.proj.weight]                          
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


In [2]:
from deepeval.models.base_model import DeepEvalBaseLLM
import torch
import re

class QwenJudge(DeepEvalBaseLLM):

    def __init__(self, model, processor):
        self.model = model
        self.processor = processor
        self.name = "Qwen2.5-VL-7B-Judge"

    def load_model(self):
        return self.model

    # RAGAS is TEXT-ONLY
    def supports_multimodal(self) -> bool:
        return True

    @staticmethod
    def _force_json(text: str) -> str:
        """
        Extract the first valid JSON object from text.
        Hard fail if none found.
        """
        match = re.search(r"\{[\s\S]*\}", text)
        if not match:
            raise ValueError(f"Judge did not output JSON:\n{text}")
        return match.group(0)

    def generate(self, prompt: str) -> str:
        system_prompt = (
            "You are a strict JSON generator.\n"
            "Return ONLY valid JSON.\n"
            "No explanations.\n"
            "No markdown.\n"
            "No trailing text."
        )

        messages = [
            {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
            {"role": "user", "content": [{"type": "text", "text": prompt}]}
        ]

        text = self.processor.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = self.processor(
            text=[text],
            padding=True,
            return_tensors="pt"
        ).to(self.model.device)

        with torch.no_grad():
            generated_ids = self.model.generate(
                **inputs,
                max_new_tokens=15000,
                do_sample=False,      # 🚨 REQUIRED
                temperature=0.0,      # 🚨 REQUIRED
                top_p=1.0,
            )

        output_ids = generated_ids[:, inputs.input_ids.shape[1]:]
        raw_output = self.processor.batch_decode(
            output_ids,
            skip_special_tokens=True
        )[0].strip()

        return self._force_json(raw_output)

    def get_model_name(self):
        return self.name

    async def a_generate(self, prompt: str) -> str:
    # Force sync execution even if DeepEval tries async
        return self.generate(prompt)

# Re-initialize the judge
local_judge = QwenJudge(model, processor)


In [3]:
import ast
import json
from deepeval.test_case import LLMTestCase, MLLMImage
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

# 1. Load your JSON
with open('RAGQwenVL7B_predictions_output.json', 'r') as f:
    data = json.load(f)

test_cases = []
for key, item in data.items():
    image_list_raw = item["question"]["ImageList"]
    
    # Standardize image list
    if isinstance(image_list_raw, str):
        image_urls = ast.literal_eval(image_list_raw)
    else:
        image_urls = image_list_raw

    # Create MLLMImage objects
    images = [MLLMImage(url=url) for url in image_urls]
    
    # 2. Use the standard LLMTestCase but include images in the input
    # This 'slug' approach is highly compatible with Qwen2.5-VL
    input_content = item["question"]["Text"]
    for i, img in enumerate(images):
        input_content += f"\nImage {i+1}: {img}"

    test_case = LLMTestCase(
        input=input_content,
        actual_output=item.get("answer", ""), 
        expected_output=item["ground_truth"],
        retrieval_context=[item["contexts"]]
    )
    test_cases.append(test_case)

print(f"Prepared {len(test_cases)} test cases.")

Prepared 16 test cases.


In [4]:
# Define chunk size
chunk_size = 2

# Partition the list into sub-lists
sub_test_cases = [test_cases[i : i + chunk_size] for i in range(0, len(test_cases), chunk_size)]

# Verification
print(f"Created {len(sub_test_cases)} sub-test cases.")
for idx, sub_list in enumerate(sub_test_cases):
    print(f"Sub-case {idx + 1} contains {len(sub_list)} samples.")

Created 8 sub-test cases.
Sub-case 1 contains 2 samples.
Sub-case 2 contains 2 samples.
Sub-case 3 contains 2 samples.
Sub-case 4 contains 2 samples.
Sub-case 5 contains 2 samples.
Sub-case 6 contains 2 samples.
Sub-case 7 contains 2 samples.
Sub-case 8 contains 2 samples.


In [5]:
from deepeval.metrics import (
    FaithfulnessMetric,
    AnswerRelevancyMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric
)

metrics = [
    FaithfulnessMetric(threshold=0.3, model=local_judge, async_mode=False, truths_extraction_limit=50),
    AnswerRelevancyMetric(threshold=0.3, model=local_judge, async_mode=False),
    ContextualPrecisionMetric(threshold=0.3, model=local_judge, async_mode=False),
    ContextualRecallMetric(threshold=0.3, model=local_judge, async_mode=False),
]


In [6]:
from deepeval import evaluate
from deepeval.evaluate import DisplayConfig

display_config = DisplayConfig(
    print_results=True
)

results = evaluate(
    test_cases=sub_test_cases[0],
    metrics=metrics,
    display_config=display_config
)


✨ You're running DeepEval's latest Faithfulness Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

/workspace/eval_env/lib/python3.12/site-packages/rich/live.py:260: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.




Metrics Summary

  - ✅ Faithfulness (score: 0.75, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.75 because the actual output contradicts the retrieval context by suggesting symptoms consistent with Ebola or Marburg virus disease rather than malaria., error: None)
  - ✅ Answer Relevancy (score: 0.875, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.88 because the image content does not provide any information about the patient's diagnosis or treatment., error: None)
  - ✅ Contextual Precision (score: 1.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 1.00 because the retrieval context provides a detailed description of a patient's symptoms and medical history, including the context of working in an Ebola treatment unit in Liberia during an active outbreak, which aligns with the expected output., error: None)
  - ✅ Contextual Recall (score: 1.0, thres

⚠ WARNING: No hyperparameters logged.
» ]8;id=149341;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 42.73s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 100.0% | Passed: 2 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [7]:
from deepeval import evaluate
from deepeval.evaluate import DisplayConfig

display_config = DisplayConfig(
    print_results=True
)

results = evaluate(
    test_cases=sub_test_cases[1],
    metrics=metrics,
    display_config=display_config
)


✨ You're running DeepEval's latest Faithfulness Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...



Metrics Summary

  - ✅ Faithfulness (score: 0.6666666666666666, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.67 because the actual output includes specific medical conditions like petechial rash, purpuric non-blanching lesions, and differential diagnoses such as malaria, enteric fever, dengue fever, and melioidosis, which are not present in the retrieval context., error: None)
  - ✅ Answer Relevancy (score: 0.4444444444444444, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.44 because the input does not mention petechial rash or purpuric non-blanching lesions, making these irrelevant statements. The score cannot be higher due to the lack of relevance to the provided medical history., error: None)
  - ✅ Contextual Precision (score: 1.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 1.00 because the retrieval context correctly identifies Dengue as th

⚠ WARNING: No hyperparameters logged.
» ]8;id=666107;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 62.96s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 50.0% | Passed: 1 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [8]:
from deepeval import evaluate
from deepeval.evaluate import DisplayConfig

display_config = DisplayConfig(
    print_results=True
)

results = evaluate(
    test_cases=sub_test_cases[2],
    metrics=metrics,
    display_config=display_config
)


✨ You're running DeepEval's latest Faithfulness Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...



Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 1.00 because there are no contradictions found between the actual output and the retrieval context., error: None)
  - ✅ Answer Relevancy (score: 0.5, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.50 because the provided JSON does not contain any relevant information about the patient's condition or the images mentioned. It lacks specific details that could be used to address the input query., error: None)
  - ❌ Contextual Precision (score: 0.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.00 because the retrieval context does not mention any symptoms or conditions related to Marburg virus disease, cerebral malaria, Knowlesi malaria, severe falciparum malaria, severe dengue, yellow fever, or invasive non-typhoidal Salmonella. It also does not provide inf

⚠ WARNING: No hyperparameters logged.
» ]8;id=443449;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 59.19s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 2

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [9]:
from deepeval import evaluate
from deepeval.evaluate import DisplayConfig

display_config = DisplayConfig(
    print_results=True
)

results = evaluate(
    test_cases=sub_test_cases[3],
    metrics=metrics,
    display_config=display_config
)


✨ You're running DeepEval's latest Faithfulness Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...



Metrics Summary

  - ❌ Faithfulness (score: 0.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.00 because the actual output contradicts the retrieval context by mentioning nematode worms and a surgical procedure for removing them, which are not present in the retrieval context., error: None)
  - ✅ Answer Relevancy (score: 1.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 1.00 because the provided JSON does not contain any irrelevant statements. All information is relevant to the input., error: None)
  - ❌ Contextual Precision (score: 0.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.00 because the retrieval context does not mention any information related to ocular onchocerciasis or nematode worms in the eye, which is the focus of the expected output., error: None)
  - ❌ Contextual Recall (score: 0.0, threshold: 0.3, strict: False, evaluation mo

⚠ WARNING: No hyperparameters logged.
» ]8;id=160936;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 40.06s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 2

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [10]:
from deepeval import evaluate
from deepeval.evaluate import DisplayConfig

display_config = DisplayConfig(
    print_results=True
)

results = evaluate(
    test_cases=sub_test_cases[4],
    metrics=metrics,
    display_config=display_config
)


✨ You're running DeepEval's latest Faithfulness Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...



Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 1.00 because there are no contradictions found between the actual output and the retrieval context., error: None)
  - ✅ Answer Relevancy (score: 1.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 1.00 because there are no irrelevant statements in the actual output., error: None)
  - ❌ Contextual Precision (score: 0.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.00 because the retrieval context does not mention any information related to trachoma, dry eye disease, or corneal ulcers. It discusses various diseases and their symptoms but does not provide any details about the specific case described in the input., error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 1.00 

⚠ WARNING: No hyperparameters logged.
» ]8;id=866907;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 45.11s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 50.0% | Passed: 1 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [11]:
from deepeval import evaluate
from deepeval.evaluate import DisplayConfig

display_config = DisplayConfig(
    print_results=True
)

results = evaluate(
    test_cases=sub_test_cases[5],
    metrics=metrics,
    display_config=display_config
)


✨ You're running DeepEval's latest Faithfulness Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...



Metrics Summary

  - ❌ Faithfulness (score: 0.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.00 because the actual output contradicts the retrieval context by describing cutaneous leishmaniasis symptoms and diagnostic methods differently than what was mentioned, leading to a lack of alignment., error: None)
  - ✅ Answer Relevancy (score: 1.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 1.00 because there are no irrelevant statements in the actual output., error: None)
  - ✅ Contextual Precision (score: 1.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 1.00 because the retrieval context directly discusses cutaneous leishmaniasis, which matches the expected output. There are no irrelevant nodes, so the contextual precision score is at its highest., error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.3, strict: False, evaluation model: 

⚠ WARNING: No hyperparameters logged.
» ]8;id=164385;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 44.36s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 2

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [12]:
from deepeval import evaluate
from deepeval.evaluate import DisplayConfig

display_config = DisplayConfig(
    print_results=True
)

results = evaluate(
    test_cases=sub_test_cases[6],
    metrics=metrics,
    display_config=display_config
)


✨ You're running DeepEval's latest Faithfulness Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...



Metrics Summary

  - ✅ Faithfulness (score: 0.5714285714285714, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.57 because the actual output contradicts the retrieval context by suggesting Actinomycetoma, which is not mentioned in the context, and also by claiming the presence of bacteria in biopsies with specific staining properties, which are not discussed in the retrieval context., error: None)
  - ✅ Answer Relevancy (score: 0.4444444444444444, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.44 because the provided JSON does not contain any irrelevant statements. The content is relevant to the input and provides necessary medical details., error: None)
  - ❌ Contextual Precision (score: 0.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.00 because the retrieval context does not mention any information about actinomycetoma or its symptoms, which a

⚠ WARNING: No hyperparameters logged.
» ]8;id=983483;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 50.3s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 2

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [15]:
from deepeval import evaluate
from deepeval.evaluate import DisplayConfig

display_config = DisplayConfig(
    print_results=True
)

results = evaluate(
    test_cases=sub_test_cases[7],
    metrics=metrics,
    display_config=display_config
)


✨ You're running DeepEval's latest Faithfulness Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using Qwen2.5-VL-7B-Judge, strict=False, 
async_mode=True)...



Metrics Summary

  - ❌ Faithfulness (score: 0.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.00 because the actual output includes references to Chronic Hepatitis C and HCV RNA, which are not mentioned in the retrieval context., error: None)
  - ✅ Answer Relevancy (score: 0.6, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.60 because the statement does not provide new information about the patient's condition and suggests a possible false-positive result, which is not directly relevant to the diagnosis., error: None)
  - ❌ Contextual Precision (score: 0.0, threshold: 0.3, strict: False, evaluation model: Qwen2.5-VL-7B-Judge, reason: The score is 0.00 because the retrieval context does not mention any information related to the patient's condition, symptoms, or laboratory findings described in the expected output. It focuses on different cases with various symptoms and diagnoses unrelated to

⚠ WARNING: No hyperparameters logged.
» ]8;id=846943;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 55.2s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 2

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.